# Analysis of the Turbofan dataset

This notebook is an attempt at studying the Turbofan dataset from NASA. The dataset and the accompanying documentation is available here: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan

The aim of the dataset is to develop algorithms that are able to predict the failure of turbofan engines in advance. In the following code, we will try to do that using *Recurrent Neural Networks* built using PyTorch (see https://www.pytorch.org) and the FastAI library (see https://docs.fast.ai/).

## Setting-up the environment and getting the file

In [1]:
# Import of the required libraries
import numpy as np
import pandas as pd
import torch
import fastai
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import urllib
from pathlib import Path
import zipfile

from pylab import mpl, plt
import seaborn

seaborn.set()

In [2]:
url = 'https://ti.arc.nasa.gov/c/6/'
CMAPSSDATAFILE = Path.cwd() / 'data' / 'CMAPSSData.zip'
DATAPATH = Path.cwd() / 'data'

In [3]:
if not CMAPSSDATAFILE.exists():
    dir = DATAPATH.mkdir(parents=True)
    filedata = urllib.request.urlretrieve(url, CMAPSSDATAFILE)
    file = zipfile.ZipFile(CMAPSSDATAFILE, 'r')
    file.extractall(CMAPSSDATAFILE.parent)
    file.close()

In [4]:
# Load the Dataset Turbofan engine
file = DATAPATH / 'train_FD001.txt'

# Let's define the column names for this dataframe
columns = ['Unit', 'Cycle', 'os1', 'os2', 'os3']

for i in range(1, 22):
    columns.append('sm'+str(i))

cols = []
for i in range(0,26):
    cols.append(i)
    
# os stands for operational setting
# sm stands for sensor measurument

data = pd.read_csv(file, names = columns, header = None, usecols = cols, sep=' ')

In [5]:
# Let's take a look at the beginning of the data and see if it's properly loaded
data.head()

Unit  Cycle     os1     os2    os3     sm1     sm2      sm3      sm4  \
0     1      1 -0.0007 -0.0004  100.0  518.67  641.82  1589.70  1400.60   
1     1      2  0.0019 -0.0003  100.0  518.67  642.15  1591.82  1403.14   
2     1      3 -0.0043  0.0003  100.0  518.67  642.35  1587.99  1404.20   
3     1      4  0.0007  0.0000  100.0  518.67  642.35  1582.79  1401.87   
4     1      5 -0.0019 -0.0002  100.0  518.67  642.37  1582.85  1406.22   

     sm5  ...    sm12     sm13     sm14    sm15  sm16  sm17  sm18   sm19  \
0  14.62  ...  521.66  2388.02  8138.62  8.4195  0.03   392  2388  100.0   
1  14.62  ...  522.28  2388.07  8131.49  8.4318  0.03   392  2388  100.0   
2  14.62  ...  522.42  2388.03  8133.23  8.4178  0.03   390  2388  100.0   
3  14.62  ...  522.86  2388.08  8133.83  8.3682  0.03   392  2388  100.0   
4  14.62  ...  522.19  2388.04  8133.80  8.4294  0.03   393  2388  100.0   

    sm20     sm21  
0  39.06  23.4190  
1  39.00  23.4236  
2  38.95  23.3442  
3  38.88  23.3739  
4  38.90  23.4044  

[5 rows x 26 columns]

In [6]:
# Let's have a quick look at statistics describing the different parameters
data.describe()

Unit         Cycle           os1           os2      os3  \
count  20631.000000  20631.000000  20631.000000  20631.000000  20631.0   
mean      51.506568    108.807862     -0.000009      0.000002    100.0   
std       29.227633     68.880990      0.002187      0.000293      0.0   
min        1.000000      1.000000     -0.008700     -0.000600    100.0   
25%       26.000000     52.000000     -0.001500     -0.000200    100.0   
50%       52.000000    104.000000      0.000000      0.000000    100.0   
75%       77.000000    156.000000      0.001500      0.000300    100.0   
max      100.000000    362.000000      0.008700      0.000600    100.0   

                sm1           sm2           sm3           sm4           sm5  \
count  2.063100e+04  20631.000000  20631.000000  20631.000000  2.063100e+04   
mean   5.186700e+02    642.680934   1590.523119   1408.933782  1.462000e+01   
std    6.537152e-11      0.500053      6.131150      9.000605  3.394700e-12   
min    5.186700e+02    641.210000   1571.040000   1382.250000  1.462000e+01   
25%    5.186700e+02    642.325000   1586.260000   1402.360000  1.462000e+01   
50%    5.186700e+02    642.640000   1590.100000   1408.040000  1.462000e+01   
75%    5.186700e+02    643.000000   1594.380000   1414.555000  1.462000e+01   
max    5.186700e+02    644.530000   1616.910000   1441.490000  1.462000e+01   

       ...          sm12          sm13          sm14          sm15  \
count  ...  20631.000000  20631.000000  20631.000000  20631.000000   
mean   ...    521.413470   2388.096152   8143.752722      8.442146   
std    ...      0.737553      0.071919     19.076176      0.037505   
min    ...    518.690000   2387.880000   8099.940000      8.324900   
25%    ...    520.960000   2388.040000   8133.245000      8.414900   
50%    ...    521.480000   2388.090000   8140.540000      8.438900   
75%    ...    521.950000   2388.140000   8148.310000      8.465600   
max    ...    523.380000   2388.560000   8293.720000      8.584800   

               sm16          sm17     sm18     sm19          sm20  \
count  2.063100e+04  20631.000000  20631.0  20631.0  20631.000000   
mean   3.000000e-02    393.210654   2388.0    100.0     38.816271   
std    1.556432e-14      1.548763      0.0      0.0      0.180746   
min    3.000000e-02    388.000000   2388.0    100.0     38.140000   
25%    3.000000e-02    392.000000   2388.0    100.0     38.700000   
50%    3.000000e-02    393.000000   2388.0    100.0     38.830000   
75%    3.000000e-02    394.000000   2388.0    100.0     38.950000   
max    3.000000e-02    400.000000   2388.0    100.0     39.430000   

               sm21  
count  20631.000000  
mean      23.289705  
std        0.108251  
min       22.894200  
25%       23.221800  
50%       23.297900  
75%       23.366800  
max       23.618400  

[8 rows x 26 columns]

## Cleaning and preparing the dataset

### Cleaning

As we can see in the dataset above, some of the parameters have a standard deviation of 0.0. This means, the values of those parameters never change. So, we can get rid of those parameters entirely in our model since they won't have any impact in our prediction. 

In [7]:
# .iloc[2,:] gives us the std row in the describe() call above
criteria = (data.describe().iloc[2,:] != 0)
data = data[criteria.index[criteria]]
data.shape

(20631, 23)

### Preparing

Now that we have remove the parameters (i.e features) that have no value to our prediction, we have to define the target variable that we will need to predict using our model. 
Reading the accompanying file with the dataset, we can see that the value we are trying to predict is the *Remaining Useful Life*, also named *RUL*. The different units in the dataset are brought to failure, therefore the *RUL* can be define with the following calculation: $RUL = Cycle_{max} - Cycle_{current}$. Let's use this formula to add a column containing *RUL* 

In [8]:
data['RUL'] = data.groupby('Unit')['Cycle'].transform(max) - data['Cycle']